# SETUP

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
!pip install vaderSentiment
!pip uninstall spacy
!pip install spacy
!python -m spacy info
# redemarrer le runtime apres

!spacy download en_core_web_trf
# !spacy download en_core_web_lg

# spacy.load("en_core_web_sm", disable=["tagger", "parser"])
#### disable useless spacy components for performance 
#### https://spacy.io/usage/processing-pipelines#disabling

import pandas as pd
import numpy as np
import nltk
import time
import spacy
sp = spacy.load('en_core_web_trf') # , disable=[""]
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
import nltk
import re

     |████████████████████████████████| 133kB 4.8MB/s 
Uninstalling spacy-2.2.4:
  Would remove:
    /usr/local/bin/spacy
    /usr/local/lib/python3.7/dist-packages/bin/*
    /usr/local/lib/python3.7/dist-packages/spacy-2.2.4.dist-info/*
    /usr/local/lib/python3.7/dist-packages/spacy/*
  Would not remove (might be manually added):
    /usr/local/lib/python3.7/dist-packages/bin/theano_cache.py
    /usr/local/lib/python3.7/dist-packages/bin/theano_nose.py
Proceed (y/n)? y
  Successfully uninstalled spacy-2.2.4
     |████████████████████████████████| 12.8MB 256kB/s 
     |████████████████████████████████| 51kB 4.9MB/s 
     |████████████████████████████████| 9.1MB 16.0MB/s 
     |████████████████████████████████| 460kB 33.7MB/s 
     |████████████████████████████████| 1.1MB 26.6MB/s 
     |████████████████████████████████| 122kB 42.4MB/s 
  Created wheel for smart-open: filename=smart_open-3.0.0-cp37-none-any.whl size=107098 sha256=f87dfedb39945a58e0b5d26e8978f82fb11c5576455176f0c561c46

In [8]:
!spacy download en_core_web_lg
sp = spacy.load('en_core_web_lg')

2021-05-11 09:10:01.070696: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
     |████████████████████████████████| 778.8MB 18kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [3]:
!python -m spacy info


============================== Info about spaCy ==============================

spaCy version    2.2.4                         
Location         /usr/local/lib/python3.7/dist-packages/spacy
Platform         Linux-5.4.109+-x86_64-with-Ubuntu-18.04-bionic
Python version   3.7.10                        
Models           en                            



In [ ]:

# !nvidia-smi


[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 9585951368995221154, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14674281152
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 10352637737517138979
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"]

# Load Datasets


## Steam

In [ ]:
# wdir="/content/drive/MyDrive/TFE/Code/steam/"

# full_df = pd.read_csv(wdir+"train.csv") 
# # Preview the first 5 lines of the loaded data 
# full_df.head()
# full_df['game_title']=full_df['title']
# full_df['review_id']=full_df['review_id']
# full_df['review_text']=full_df['user_review']
# full_df['score']=full_df['user_suggestion']
# full_df=full_df[["review_id", "game_title", "review_text","score"]]
# full_df.groupby('game_title').review_id.nunique()
# used_games=["Eternal Card Game",
# "Fractured Spacy",
# "Heroes & Generals",
# "Robocraft",
# "Bless Online",
# "SMITE®"]
# used_games_dict = {i: used_games[i] for i in range(0, len(used_games))}  
# print(used_games_dict)   
# df=full_df
# df=full_df.loc[df['game_title'].isin(used_games)]
# print("df shape: ", df.shape)
# df.to_pickle(wdir+"1_basic_df.pkl")
# df.head()

## metacritic

In [ ]:
# wdir="/content/drive/MyDrive/TFE/Code/metacritic/"

# full_df = pd.read_csv(wdir+"metacritic_critic_reviews.csv") 
# # Preview the first 5 lines of the loaded data 
# full_df.head()
# full_df=full_df.drop(['name', 'date'], axis=1)
# full_df = full_df[full_df.platform == "PC"]
# full_df=full_df.drop(['platform'], axis=1)
# full_df = full_df.sample(frac=1).reset_index(drop=False) # random sampling and resetting index
# full_df['game_title']=full_df['game']
# full_df['review_id']=full_df['index']
# full_df['review_text']=full_df['review']

# full_df=full_df[["review_id", "game_title", "review_text","score"]]

# used_games=["Company of Heroes 2",      ## games with most reviews ( around 100 each ) in the dataset     
# "Diablo III",                     
# "Sid Meier's Civilization VI",   
# "SUPERHOT",                       
# "XCOM 2"]       
# used_games_dict = {i: used_games[i] for i in range(0, len(used_games))}  
# print(used_games_dict)            
# df=full_df
# df=df.loc[df['game_title'].isin(used_games)].reset_index(drop=True)
# print("df shape: ", df.shape)
# df.to_pickle(wdir+"1_basic_df.pkl")

# df.head()

## AM Videogames


In [41]:
# !pip3 install pickle5
# import pickle5 as pickle
wdir="/content/drive/MyDrive/TFE/Code/AM_videogames/"
# with open(wdir+"1_basic_df.pkl", "rb") as fh:
#   df = pickle.load(fh)

df = pd.read_pickle(wdir+"1_basic_df.pkl")

used_games=["diablo III",  ## games with most reviews ( around 100 each ) in the dataset     
"God of War III",     
"Mario Kart",                     
"The Last of Us",   
"StarCraft II: Wings of Liberty"]   

used_games_dict = {i: used_games[i] for i in range(0, len(used_games))}  
print(used_games_dict)            

df=df.loc[df['game_title'].isin(used_games)].reset_index(drop=True)
print("df shape: ", df.shape)
df.to_pickle(wdir+"1_basic_df.pkl")
df.head()

{0: 'diablo III', 1: 'God of War III', 2: 'Mario Kart', 3: 'The Last of Us', 4: 'StarCraft II: Wings of Liberty'}
df shape:  (4677, 5)


,review_id,game_title,summary,review_text,score
0,123125,Mario Kart,My 4 1/2 year old LOVES this,I would have thought that he was too young to ...,5.0
1,126176,God of War III,Much better than GOW I and II,"This game is a perfect squeal/""season finale""....",4.0
2,149896,StarCraft II: Wings of Liberty,An Epic Sequel to an Epic Game!,"In a nutshell, this game is more or less of th...",5.0
3,125482,StarCraft II: Wings of Liberty,"Looks nice, but who cares.",Blizzard is now my enemy. The great quality of...,2.0
4,126316,God of War III,Great Game ... but Expected More From the Finale,This is an descent experience but not quite wo...,3.0


# data Visualization

In [ ]:
print(df.groupby('game_title').review_id.nunique())
# df = df.sample(frac=1).reset_index(drop=True) # random sampling and resetting index


game_title
God of War III                     905
Mario Kart                         834
StarCraft II: Wings of Liberty     774
The Last of Us                     783
diablo III                        1381
Name: review_id, dtype: int64


# spacy extractor

In [15]:
import pickle

# POSLemmaChoice="SP" # WN TB SP
chosen_game="The Last of Us"
# "diablo III"
# "God of War III"   
# "Mario Kart"                   
# "The Last of Us"  
# "StarCraft II: Wings of Liberty" 
basic_df = pd.read_pickle(wdir+"1_basic_df.pkl")
basic_df=basic_df.loc[basic_df["game_title"]== chosen_game]
 
basic_df=basic_df.reset_index(drop=False)
basic_df["review_id"]=basic_df["index"]
basic_df=basic_df.drop(["index"], axis=1)



df_len=len(basic_df.index)
print(df_len)
percent=int(df_len/100)
basic_df.head()
i=0
doc_list=[]
for index, row in basic_df.iterrows():
  # i+=1
  # if i>1: break
  if (index%percent)==1:
    print(int(index/df_len*100) , "   % processed")
  doc = sp(row["review_text"])
  doc_list.append(doc)

''' Serialization '''
# Serialize vocab (actually the whole NLP ojbect)
pickle.dump(doc_list, open(wdir+chosen_game+"_SP.pickle", "wb"))

# ''' Deserialization '''
# reloaded_doc_list = pickle.load(open(wdir+chosen_game+"_SP.pickle", "rb"))


783
0    % processed
1    % processed
1    % processed
2    % processed
3    % processed
4    % processed
5    % processed
6    % processed
7    % processed
8    % processed
9    % processed
9    % processed
10    % processed
11    % processed
12    % processed
13    % processed
14    % processed
15    % processed
16    % processed
17    % processed
18    % processed
18    % processed
19    % processed
20    % processed
21    % processed
22    % processed
23    % processed
24    % processed
25    % processed
26    % processed
26    % processed
27    % processed
28    % processed
29    % processed
30    % processed
31    % processed
32    % processed
33    % processed
34    % processed
34    % processed
35    % processed
36    % processed
37    % processed
38    % processed
39    % processed
40    % processed
41    % processed
42    % processed
43    % processed
43    % processed
44    % processed
45    % processed
46    % processed
47    % processed
48    % processed
49    % processed


In [ ]:
''' Deserialization '''
loaded_doc_list = pickle.load(open(wdir+chosen_game+"_SP.pickle", "rb"))
print(len(loaded_doc_list))

783


# Preprocessing reviews

## split reviews in sentences + compute sentence polarities
try textblob and nltk too for polarity. 

Vader Sentiment polarity ( grabs negation ) 

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyser = SentimentIntensityAnalyzer()
def getVaderPolarity(sentence):
  return analyser.polarity_scores(sentence)["compound"]

def getVaderMatch(df):
  df["vader_match"]=np.where(  
      ( ((df['score'] > 0.5) & (df['vader_polarity'] > 0)) |  ((df['score'] < 0.5) & (df['vader_polarity'] < 0)) ) 
        , 1, 0)
  return df
  
## NLTK sentiment 

# textblob sentiment



In [ ]:
from nltk import tokenize # del
nltk.download('punkt')

df = pd.read_pickle(wdir+"1_basic_df.pkl")

tic= time.time()

i=0
j=0
df_len=len(df.index)
print(df_len)
percent=int(df_len/10)
sentences_dic={}
for index, row in df.iterrows():
  i+=1
  # if i>200:
  #   break
  if (i%percent)==1:
    print(int(i/df_len*100) , "   % processed")
  sentences = nltk.tokenize.sent_tokenize(row["review_text"])
  
  for sentence in sentences:
    
    sentences_dic[j] = {'review_id' : row["review_id"],
                    'game_title' : row["game_title"],
                    'sentence': sentence}
    j+=1
    
sentences_df = pd.DataFrame.from_dict(sentences_dic,orient='index')  # Creation of the dataframe   
sentences_df = sentences_df.sort_index()  # sorting by index

toc= time.time() - tic
print("split in sentences processed in " , toc, " seconds")

tic= time.time()
sentences_df['vader_polarity'] = sentences_df["sentence"].apply(getVaderPolarity)
toc= time.time() - tic
print("vader processed in " , toc, " seconds")

print(sentences_df.shape)
sentences_df.to_pickle(wdir+"2_sentences_df.pkl")

sentences_df.head()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
4677
0    % processed
10    % processed
19    % processed
29    % processed
39    % processed
49    % processed
59    % processed
69    % processed
79    % processed
89    % processed
99    % processed
split in sentences processed in  2.366905450820923  seconds
vader processed in  5.2729737758636475  seconds
(47525, 4)


,review_id,game_title,sentence,vader_polarity
0,123125,Mario Kart,I would have thought that he was too young to ...,0.3064
1,123125,Mario Kart,"However, there are new players for him to earn...",0.0000
2,123125,Mario Kart,"You do have to unlock many of the courses, but...",0.5499
3,123125,Mario Kart,"The courses are fairly intuitive, so even when...",0.6422
4,123125,Mario Kart,A fun game for the whole family.,0.5106


## Stop words


In [ ]:
from nltk.corpus import stopwords # del
import nltk # del
nltk.download('stopwords')
stop_words = stopwords.words('english')
import re # del
""""""

other_stop_words="""
game games play playing playin played
a	hereupon	six	differently
a’s	hers	so	downed
able	herself	some	downing
about	hi	somebody	downs
above	him	somehow	early
according	himself	someone	end
accordingly	his	something	ended
across	hither	sometime	ending
actually	hopefully	sometimes	ends
after	how	somewhat	evenly
afterwards	howbeit	somewhere	face
again	however	soon	faces
against	i	sorry	fact
ain’t	i’d	specified	facts
all	i’ll	specify	felt
allow	i’m	specifying	find
allows	i’ve	still	finds
almost	ie	sub	full
alone	if	such	fully
along	ignored	sup	furthered
already	immediate	sure	furthering
also	in	t	furthers
although	inasmuch	t’s	gave
always	inc	take	general
am	indeed	taken	generally
among	indicate	tell	give
amongst	indicated	tends
an	indicates	th
and	inner	than
another	insofar	thank
any	instead	thanks
anybody	into	thanx	group
anyhow	inward	that	grouped
anyone	is	that’s	grouping
anything	isn’t	thats	groups
anyway	it	the
anyways	it’d	their
anywhere	it’ll	theirs
apart	it’s	them
appear	its	themselves
appreciate	itself	then
appropriate	j	thence
are	just	there
aren’t	k	there’s	kind
around	keep	thereafter	knew
as	keeps	thereby
aside	kept	therefore
ask	know	therein	latest
asking	knows	theres	lets
associated	known	thereupon	long
at	l	these	longer
available	last	they
away	lately	they’d	made
awfully	later	they’ll	make
b	latter	they’re	making
be	latterly	they’ve	man
became	least	think
because	less	third
become	lest	this	men
becomes	let	thorough	mr
becoming	let’s	thoroughly	mrs
been	like	those	needed
before	liked	though	needing
beforehand	likely	three
behind	little	through
being	look	throughout	number
believe	looking	thru	numbers
below	looks	thus
beside	ltd	to
besides	m	together	open	mainly	too	opened	many	took	opening
between	may	toward	opens
beyond	maybe	towards	order
both	me	tried
brief	mean	tries
but	meanwhile	truly
by	merely	try	part
c	might	trying	parted
c’mon	more	twice	parting
c’s	moreover	two	parts
came	most	u
can	mostly	un
can’t	much	under	point
cannot	must	unfortunately	pointed
cant	my	unless	pointing
cause	myself	unlikely	points
causes	n	until	present
certain	unto	presented
certainly	up	presenting
changes	nd	upon	presents
clearly	near	us
co	nearly	use
com	used	put
come	useful	puts
comes	uses	room
concerning	neither	using	rooms
consequently	never	usually	seconds
consider	nevertheless	uucp	sees
considering	v	show
contain	next	showed
containing	nine	various	showing
contains	no	very	shows
corresponding	nobody	via	side
could	non	viz	sides
couldn’t	none	vs
course	noone	w
currently	nor	want
d	normally	wants	state
definitely	not	was	states
described	nothing	wasn’t	thing
despite	novel	way	things
did	now	we	thinks
didn’t	nowhere	we’d	thought
different	o	we’ll	thoughts
do	obviously	we’re	today
does	of	we’ve	turn
doesn’t	off	welcome	turned
doing	often	well	turning
don’t	oh	went	turns
done	ok	were	wanted
down	okay	weren’t	wanting
downwards	old	what	ways
during	on	what’s	wells
e	once	whatever
each	one	when
edu	ones	whence
eg	only	whenever
eight	onto	where	year
either	or	where’s	years
else	other	whereafter	young
elsewhere	others	whereas	younger
enough	otherwise	whereby	youngest
entirely	ought	wherein	beings
especially	our	whereupon	big
et	ours	wherever	case
etc	ourselves	whether	cases
even	out	which	clear
ever	outside	while	differ
every	over	whither	hence
everybody	overall	who	her
everyone	own	who’s	here
everything	p	whoever	here’s
everywhere	particular	whole	hereafter
ex	particularly	whom	hereby
exactly	per	whose	herein
example	perhaps	why	seven
except	placed	will	several
f	please	willing	shall
far	plus	wish	she
few	possible	with	should
fifth	presumably	within	shouldn’t
first	probably	without	since
five	provides	won’t	haven’t
followed	q	wonder	having
following	que	would	he
follows	quite	wouldn’t	he’s
for	qv	x	hello
former	r	y	help
formerly	rather	yes	self
forth	rd	yet	selves
four	re	you	sensible
from	really	you’d	sent
further	reasonably	you’ll
furthermore	regarding	you’re
g	regardless	you’ve	asks
get	regards	your	back
gets	relatively	yours	backed
getting	respectively	yourself	backing
given	right	yourselves	backs
gives	s	z	began
go	said	zero
goes	same	he’d	has
going	saw	he’ll	hasn’t
gone	say	how’s	have
got	saying	mustn’t	seemed
gotten	says	ours 	seeming
greetings	second	shan’t	seems
h	secondly	she’d	seen
had	see	she’ll	why’s
hadn’t	seeing	she’s	area
happens	seem	when’s	asked"""

#add other stop words
for word in other_stop_words.split():
  stop_words.append(word)
print(len(stop_words))

#add words from game titles
for game in used_games:
  for word in game.split():
    # print(word.lower())
    stop_words.append(word.lower())
print(len(stop_words))


def remove_stop_words(sentence,stop_words):
  #removes common stop words, numbers and words from game titles
  word_list = re.sub("[^\w]", " ",  sentence).split() # remove non alphanumerical characters and split 
  filtered_word_list = [word for word in word_list if word not in stop_words]
  filtered_word_list = [word for word in filtered_word_list if word.isnumeric()==False]
  
  sentence = " ".join(filtered_word_list)
  return sentence


prefixes_list=["Early Access Review"]

def remove_prefixes(sentence,prefixes_list):
  for prefix in prefixes_list:
    sentence=sentence.replace( prefix, '')
  return sentence


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
865
882


In [ ]:
sentences_df = pd.read_pickle(wdir+"2_sentences_df.pkl")
sentences_df.head()


tic= time.time()
df_len=len(sentences_df.index)
print(df_len)
percent=int(df_len/10)
proc_sent_dic={}

for index, row in sentences_df.iterrows():
  # if index>200:
  #   break
  if (index%percent)==1:
    print(int(index/df_len*100) , "   % processed")

  proc_sent = remove_prefixes(row["sentence"],prefixes_list)
  proc_sent = proc_sent.lower()
  proc_sent = remove_stop_words(proc_sent,stop_words)

  proc_sent_dic[index] = {'review_id' : row["review_id"],
                  'game_title' : row["game_title"],
                  'sentence': row["sentence"],
                  "vader_polarity": row["vader_polarity"],
                  'processed_sentence': proc_sent}
    
proc_sent_df = pd.DataFrame.from_dict(proc_sent_dic,orient='index')  # Creation of the dataframe   
proc_sent_df = proc_sent_df.sort_index()  # sorting by index

toc= time.time() - tic
print("processed in " , toc, " seconds")

print(proc_sent_df.shape)
print(proc_sent_df.shape)
proc_sent_df.to_pickle(wdir+"3_proc_sent_df.pkl")
proc_sent_df.head()


47525
0    % processed
10    % processed
20    % processed
29    % processed
39    % processed
49    % processed
59    % processed
69    % processed
79    % processed
89    % processed
99    % processed
processed in  9.88982343673706  seconds
(47525, 5)
(47525, 5)


,review_id,game_title,sentence,vader_polarity,processed_sentence
0,123125,Mario Kart,I would have thought that he was too young to ...,0.3064,enjoy kid courses unlocked week
1,123125,Mario Kart,"However, there are new players for him to earn...",0.0000,new players earn involved
2,123125,Mario Kart,"You do have to unlock many of the courses, but...",0.5499,unlock courses unlocked
3,123125,Mario Kart,"The courses are fairly intuitive, so even when...",0.6422,courses fairly intuitive new completely bomb
4,123125,Mario Kart,A fun game for the whole family.,0.5106,fun family


## POS + Lemmatization
TB and WN, 30 sec for 20k sentences
SP is way longer, 60x longer (30 min )




### Wordnet POSLemma function

In [ ]:
# WORDNET LEMMATIZER (with appropriate pos tags) "v" "n" "a" "r"

from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import wordnet

lemmatizer = WordNetLemmatizer()

# simplified Wordnet Tags 
def WN_simple_tagger(nltk_tag):
	if nltk_tag.startswith('J'):  # "a"
		return wordnet.ADJ
	elif nltk_tag.startswith('V'): # "v"
		return wordnet.VERB
	elif nltk_tag.startswith('N'): # "n"
		return wordnet.NOUN
	elif nltk_tag.startswith('R'): # "r"
		return wordnet.ADV
	else:		
		return None

def WordNetPOSLemmatizer(sentence):
  # tokenize the sentence and find the POS tag for each token
  pos_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))
  # print(pos_tagged)
  wordnet_tagged = list(map(lambda x: (x[0], WN_simple_tagger(x[1])), pos_tagged))
  # print(wordnet_tagged)
  lemmatized_sentence = []
  for word, tag in wordnet_tagged:
    if tag is None: # if there is no available tag, append the token as is
      lemmatized_sentence.append(word)
    else: # else use the tag to lemmatize the token
      lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
  lemmatized_sentence = " ".join(lemmatized_sentence) #reform the sentence
  # print(lemmatized_sentence)
  return lemmatized_sentence , wordnet_tagged

sentence = 'the cat is sitting with the bats on the striped mat under many badly flying geese'
WordNetPOSLemmatizer(sentence)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


('the cat be sit with the bat on the striped mat under many badly fly geese',
 [('the', None),
  ('cat', 'n'),
  ('is', 'v'),
  ('sitting', 'v'),
  ('with', None),
  ('the', None),
  ('bats', 'n'),
  ('on', None),
  ('the', None),
  ('striped', 'a'),
  ('mat', 'n'),
  ('under', None),
  ('many', 'a'),
  ('badly', 'r'),
  ('flying', 'v'),
  ('geese', 'a')])

### TextBlob POSLemma function

In [ ]:
from textblob import TextBlob, Word
  
def TextBlobPOSLemmatizer(sentence):
    sent = TextBlob(sentence)
    tag_dict = {"J": 'a', "N": 'n', "V": 'v', "R": 'r'}
    words_tags = [(word, tag_dict.get(pos_tag[0], None)) for word, pos_tag in sent.tags]   # convert to our simplified  word tag 
    lemma_list = [wd.lemmatize(tag) for wd, tag in words_tags] # Lemmatize using tag
    lemmatized_sentence = " ".join(lemma_list)
    return lemmatized_sentence, words_tags

sentence = 'the cat is sitting with the bats on the striped mat under many badly flying geese.'
TextBlobPOSLemmatizer(sentence)


('the cat be sit with the bat on the striped mat under many badly fly goose',
 [('the', None),
  ('cat', 'n'),
  ('is', 'v'),
  ('sitting', 'v'),
  ('with', None),
  ('the', None),
  ('bats', 'n'),
  ('on', None),
  ('the', None),
  ('striped', 'a'),
  ('mat', 'n'),
  ('under', None),
  ('many', 'a'),
  ('badly', 'r'),
  ('flying', 'v'),
  ('geese', 'n')])

### Spacy POSLemma fucntion

In [ ]:
def SpacyPOSLemmatizer(sentence):
  sentence= sp(sentence)
  lemmatized_sentence = " ".join([word.lemma_ for word in sentence ])
  tagList=[]
  tag_dict = {"ADJ": 'a', "NOUN": 'n', "VERB": 'v', "ADV": 'r'}
  for word in sentence:
    tagList.append( (word.text, tag_dict.get(word.pos_, None)) )
  return lemmatized_sentence, tagList
sentence ='Manchester United is looking to sign a forward for $90 million'
# sentence =nlp(u'Manchester United is looking to sign a forward for $90 million')

# for word in sentence:
    # print(word.text, word.pos_, word.lemma) # pos is key, pos_ is value for key=pos, etc 
sentence = 'him the cat is sitting with the bats on the striped mat under many badly flying geese.'
# sentence = 'the children, the child, the wheel, the wheels'
SpacyPOSLemmatizer(sentence)

('he the cat be sit with the bat on the striped mat under many badly fly goose .',
 [('him', None),
  ('the', None),
  ('cat', 'n'),
  ('is', None),
  ('sitting', 'v'),
  ('with', None),
  ('the', None),
  ('bats', 'n'),
  ('on', None),
  ('the', None),
  ('striped', 'a'),
  ('mat', 'n'),
  ('under', None),
  ('many', 'a'),
  ('badly', 'r'),
  ('flying', 'v'),
  ('geese', 'n'),
  ('.', None)])

### Apply PosLemma

In [ ]:
def POSLemmaFunction(POSLem_choice, sentence ):
    if POSLem_choice == "WN": return WordNetPOSLemmatizer(sentence)
    elif POSLem_choice== "TB": return TextBlobPOSLemmatizer(sentence)
    elif POSLem_choice== "SP": return SpacyPOSLemmatizer(sentence)


In [ ]:
POSLemmaChoice="SP" # WN TB SP
proc_sent_df = pd.read_pickle(wdir+"3_proc_sent_df.pkl")


proc_sent_df=proc_sent_df.loc[proc_sent_df['game_title'].isin(used_games)]
proc_sent_df=proc_sent_df.reset_index(drop=True)


tic=time.time()
df_len=len(proc_sent_df.index)
print(df_len)
percent=int(df_len/100)
POSLemma_sent_dic={}
for index, row in proc_sent_df.iterrows():


  if (index%percent)==1:
    print(int(index/df_len*100) , "   % processed")

  processed_sentence, POS_tags= POSLemmaFunction(POSLemmaChoice , row["processed_sentence"])

  POSLemma_sent_dic[index] = {'review_id' : row["review_id"],
                  'game_title' : row["game_title"],
                  'sentence': row["sentence"],
                  'vader_polarity': row["vader_polarity"],
                  'processed_sentence': processed_sentence,
                  'tags':POS_tags}
    
POSLemma_sent_df = pd.DataFrame.from_dict(POSLemma_sent_dic,orient='index')  # Creation of the dataframe    
POSLemma_sent_df = POSLemma_sent_df.sort_index()  # sorting by index

POSLemma_sent_df.shape

toc=time.time()-tic
print(toc, " seconds to process")

## Save POSLemma Dataframe
print(POSLemma_sent_df.shape)
POSLemma_sent_df.to_pickle(wdir+"4_"+POSLemmaChoice+"_sent_df.pkl")

POSLemma_sent_df.head(5)
# Features will be tagged 'n', ignore None, and others will be Sentiments

# Create FSPs and compute importance, DONE

Not handling features repeting in the same sentence yet ( dict(keys don't stack, could add a "stack_count"). 



Attention weights 

## Similarity grouping/clustering


In [ ]:
FSP_df = pd.read_pickle(wdir+"4_"+POSLemmaChoice+"_sent_df.pkl")

for index, row in FSP_df.iterrows():
  if index>5:
    break
  print(row["sentence"])

## FSPs extraction

In [ ]:
def FSPs_extractor(POS_tags, polarity, FSPs_game_dic):
  F_list=[]
  S_list=[]
  FSPs={}

  # tag features and sentiments
  for word, tag in POS_tags:
    if tag=="n":
      # print(word, tag)
      F_list.append(word)
    else: # or tag=="v"
      S_list.append(word)

  # Count FSPs and weighted importance
  for feature in F_list:
    if feature not in FSPs_game_dic:
      FSPs_game_dic[feature]={} 

    FSPs[feature]=S_list

    for sentiment in S_list:
      if sentiment not in FSPs_game_dic[feature]:
        FSPs_game_dic[feature][sentiment]={}
        FSPs_game_dic[feature][sentiment]["count"]=1 # frequency, number of occurence of the pair
        FSPs_game_dic[feature][sentiment]["importance"]=polarity # frequency weighted with polarity of the sentence containing the occurence
      else:
        FSPs_game_dic[feature][sentiment]["count"]+=1
        FSPs_game_dic[feature][sentiment]["importance"]+=polarity
      

  return FSPs, FSPs_game_dic
  # print(F_list)
# def product_FSPs_extractor(df):
#   count all FSPs 

In [ ]:
# FSP_df = pd.read_pickle(wdir+"TB_sent_df.pkl")
FSP_df = pd.read_pickle(wdir+"4_"+POSLemmaChoice+"_sent_df.pkl")

# print(FSP_df.groupby('game_title').review_id.nunique())
chosen_game=0
print(used_games_dict[chosen_game])
FSP_df=FSP_df.loc[FSP_df['game_title'] == used_games_dict[chosen_game]]
FSP_df=FSP_df.reset_index(drop=True)




tic=time.time()
df_len=len(FSP_df.index)
print(df_len)
percent=int(df_len/10)
FSPs_sent_dic={}
FSPs_game_dic={}
for index, row in FSP_df.iterrows():
  # if index>5:
  #   break
  if (index%percent)==1:
    print(int(index/df_len*100) , "   % processed")

  sent_FSPs, FSPs_game_dic = FSPs_extractor(row["tags"], row["vader_polarity"], FSPs_game_dic)
  # print(FSPs_game_dic)
  FSPs_sent_dic[index] = {'review_id' : row["review_id"],
                  'game_title' : row["game_title"],
                  'sentence': row["sentence"],
                  'vader_polarity': row["vader_polarity"],
                  'processed_sentence': row["processed_sentence"],
                  'tags': row["tags"],
                  'FSPs': sent_FSPs}
    

toc=time.time()-tic
print(toc, " seconds to process")
# FSP_df.head(50)

In [ ]:
print(used_games_dict[chosen_game])
new_FSPs={}
for feature, sentiments in FSPs_game_dic.items():
  for  sentiment, scores in sentiments.items():
    new_FSPs[feature+"_"+sentiment]=scores


FSPs_df = pd.DataFrame.from_dict(new_FSPs,orient='index',columns=["count", "importance"] )  # Creation of the dataframe 


FSPs_df["mean_polarity"] = FSPs_df["importance"]/FSPs_df["count"]
FSPs_df = FSPs_df.sort_values('count', ascending=False)
maxCount = FSPs_df.iloc[0]["count"]
FSPs_df = FSPs_df.loc[FSPs_df["count"] > maxCount/20]
FSPs_df = FSPs_df.sort_values('importance', ascending=False)
print(FSPs_df.shape)
FSPs_df.head(50)

In [ ]:
FSPs_df = FSPs_df.sort_values('importance', ascending=True)
FSPs_df.head(50)

In [ ]:
FSPs_df = FSPs_df.sort_values('importance', ascending=True)
FSPs_df.head(50)


### TRY and ignore neutral comments ( polarity threshold )  

### Clustering and visualization  


### Wordnet distance or Word2Vec embedding for cosine similarity and further feature grouping and sentiment grouping.

### handle negation 
(no need if sentence level, sentiment polarity handles it already ) 
But what if we use dependency parsing? then we need to know if the link between words is negative or not. 

### output as results simply the sentiment importance towards features, without sentiment words

### run spacy trf on whole sentences ( lowercase or not? ) or even whole documents(!!!! mean polarity doc ment wise has even less sense ) ? and delete stop words after(or during )/get lemma afterwards  

### Relationship/relation extraction

# Use sentence segmentation from spacy ( linguistic features in spacy doc ) 


# Clustering


In [48]:
import pickle
chosen_game="The Last of Us"
# chosen_game="diablo III"
''' Deserialization '''
# doc_list
# loaded_doc_list
doc_list = pickle.load(open(wdir+chosen_game+"_SP.pickle", "rb"))
print(len(doc_list))


783


In [50]:
features_list=[]
for doc in doc_list:
  for token in doc: 
    # print(token.vector)
    features_list.append(token._.trf_data)
    
features_np=np.array( features_list)
print(len(features_np))
 

70329


In [ ]:
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
import re
import nltk
import gensim
from gensim.models import word2vec
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
model = word2vec.Word2Vec(corpus, size=100, window=20, min_count=2, workers=4)
model.wv['luxurious']

In [35]:
text = "The Shiba Inu is a dog that is more like a cat."
doc1 = sp(text)
text = "The dog of the neighbour is black."
doc2 = sp(text)
loaded_doc_list= [doc1,doc2]
# np array 
features_list=[]
for doc in loaded_doc_list:
  for token in doc: 
    # print(token.vector)
    features_list.append(token.vector)
    
features_np=np.array( features_list)
print(len(features_np))

21


In [47]:
######################### train clustering model + internal indices ###########
import os
clusterMethod="kmeans" 
# clusterMethod="mshifts"
# clusterMethod="agglo" 
start=time.time()
clusteringModel, n_clusters = clusterAlgorithm(clusterMethod, features_list=features_list)

total=time.time()-start
print("time to compute : " +str(total))



start Kmeans for 2 clusters
start computing sil
sil - highest 0.33260375
start computing calinski
cal - highest 7.807129425850146
start computing davies
davies - lowest 1.3428017823587548
start Kmeans for 3 clusters
start computing sil
sil - highest 0.35625878
start computing calinski
cal - highest 7.313010863356076
start computing davies
davies - lowest 0.8570318003618538
start Kmeans for 4 clusters
start computing sil
sil - highest 0.33089912
start computing calinski
cal - highest 6.5366526577134
start computing davies
davies - lowest 0.804096540631242
start Kmeans for 5 clusters
start computing sil
sil - highest 0.351673
start computing calinski
cal - highest 6.6202542439549905
start computing davies
davies - lowest 0.70210942618581
start Kmeans for 6 clusters
start computing sil
sil - highest 0.37266526
start computing calinski
cal - highest 7.207384684147427
start computing davies
davies - lowest 0.6216186080784861
time to compute : 0.2034296989440918


In [46]:
import numpy as np
from sklearn import cluster
from sklearn import metrics

def internalIndices(labels=None, features_list=None):
    print("start computing sil")
    sil_score = metrics.silhouette_score(features_list, labels, metric='euclidean')
    print("sil - highest " + str(sil_score))
    print("start computing calinski")
    calinski_score = metrics.calinski_harabasz_score(features_list, labels)
    print("cal - highest " + str(calinski_score))
    print("start computing davies")
    davies_score = metrics.davies_bouldin_score(features_list, labels)
    print("davies - lowest " + str(davies_score))
    return list((sil_score, calinski_score, davies_score))


def Kmeans(features_list=None, n_clusters=None):
    kmeans_model = cluster.KMeans(n_clusters=n_clusters, random_state=0).fit(features_list)
    labels = kmeans_model.labels_
    sil_score, calinski_score, davies_score = internalIndices(labels=labels, features_list=features_list)
    return kmeans_model, sil_score, calinski_score, davies_score


def KMeansAnalysis(features_list=None, n_clusters_list=None):
    sil_scores = []
    calinski_scores = []
    davies_scores = []
    kmeans_models = []
    for n_clusters in n_clusters_list:
        print("start Kmeans for " + str(n_clusters) + " clusters")
        kmeans_model, sil_score, calinski_score, davies_score = Kmeans(features_list=features_list,
                                                                       n_clusters=n_clusters)
        kmeans_models.append(kmeans_model)
        sil_scores.append(sil_score)  # highest is best
        calinski_scores.append(calinski_score)  # highest is best
        davies_scores.append(davies_score)  # closest to 0 is best
    # useful code if looking for different numbers of clusters
    max_sil = max(sil_scores)
    max_cal = max(calinski_scores)
    min_dav = min(davies_scores)
    max_sil_id = sil_scores.index(max_sil)
    max_cal_id = calinski_scores.index(max_cal)
    min_dav_id = davies_scores.index(min_dav)

    return kmeans_models, sil_scores, calinski_scores, davies_scores

def count_clusters(model=None):
    labels = model.labels_
    labels_unique = np.unique(labels)
    n_clusters = len(labels_unique)
    return n_clusters


def MeanShiftsAnalysis(bandwidth=None, features_list=None):
    meanshifts_model = cluster.MeanShift(bandwidth=bandwidth).fit(features_list)
    n_clusters=count_clusters(meanshifts_model)
    return meanshifts_model, n_clusters


def AgglomerativeAnalysis(features_list=None):
    agglo_model = cluster.AgglomerativeClustering().fit(features_list)
    n_clusters = count_clusters(agglo_model)
    return agglo_model, n_clusters


def clusterAlgorithm(clusterMethod=None, features_list=None):
    if clusterMethod == "kmeans":
        clusteringModels = KMeansAnalysis(features_list=features_list, n_clusters_list=[2,3,4,5,6])
        clusteringModel = clusteringModels[0][0]
        n_clusters = 2

    elif clusterMethod == "mshifts":
        bandwidth = cluster.estimate_bandwidth(features_list, quantile=0.2, n_samples=100)
        clusteringModel, n_clusters = MeanShiftsAnalysis(bandwidth=bandwidth, features_list=features_list)

    elif clusterMethod == "agglo":
        clusteringModel, n_clusters = AgglomerativeAnalysis(features_list=features_list)

    else:
        print("uncorrect clusterMethod")

    return clusteringModel, n_clusters

In [37]:
from sklearn.decomposition import PCA
import time
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
import itertools

def hi():
    print("hi")
def run_PCA(features_list=None, n_components=None):
    time_start = time.time()
    pca = PCA(n_components=n_components)
    results = pca.fit_transform(features_list)
    # print('Explained variation per principal component: {}'.format(pca.explained_variance_ratio_))
    print('Total explained variation: {}'.format(sum(pca.explained_variance_ratio_)))
    print(results.shape)
    print('PCA done in {} seconds'.format(time.time() - time_start))
    return results


def run_TSNE(features_list=None, n_components=None):
    time_start = time.time()
    tsne = TSNE(n_components=n_components, verbose=1, perplexity=40, n_iter=300)
    results = tsne.fit_transform(features_list)
    print('t-SNE done in {} seconds'.format(time.time() - time_start))
    return results


def visualize_2D(df=None, results=None, palette=None, avgpal=None, save=None, avg=None):
    plt.figure(figsize=(16, 10))

    sns.scatterplot(
        x=results[:, 0], y=results[:, 1],
        hue="labels",
        palette=palette,
        data=df,
        legend="full",
        alpha=1
    )
    sns.scatterplot(
        x=avg[:, 0], y=avg[:, 1],
        c=avgpal,
        alpha=1,
        marker="X"
    )
    if save == True:
        print("save not implemented yet")


def colors(fl=None, ml=None, label_names=None, n_split=None):
    colorset = []
    for label in label_names:
        if label[0] == "h":
            colorset.append("green")
        if label[0] == "i":
            colorset.append("red")
        if label[0] == "o":
            colorset.append("blue")

    if fl == "all":
        flcolors = colorset
    elif isinstance(fl, str):
        flcolors = [fl] * len(label_names)
        if fl in label_names:
            flcolors = []
            for label in label_names:
                if label == fl:
                    flcolors.append("gray")
                else:
                    flcolors.append("white")

    if ml == "all":
        mlcolors = colorset
    elif isinstance(ml, str):
        mlcolors = [ml] * len(label_names)
        if fl in label_names:
            mlcolors = []
            for label in label_names:
                if label == fl:
                    mlcolors.append("gray")
                else:
                    mlcolors.append("white")

    repeated_mlcolors = list(
        itertools.chain.from_iterable(itertools.repeat(mlcolors[i], n_split[i]) for i in range(len(mlcolors))))
    return flcolors, repeated_mlcolors


def visualize_3D(results=None, labels=None, save=None):
    colors = []
    for i in labels["labels"]:
        if i[0] == "h":
            colors.append("green")
        if i[0] == "i":
            colors.append("red")
        if i[0] == "o":
            colors.append("blue")
        if i[0] == "P":
            colors.append("green")
        if i[0] == "U":
            colors.append("red")

    ax = plt.figure(figsize=(16, 10)).gca(projection='3d')
    ax.scatter(
        xs=results[:, 0], ys=results[:, 1], zs=results[:, 2],
        c=colors
    )
    ax.set_xlabel('dim-one')
    ax.set_ylabel('dim-two')
    ax.set_zlabel('dim-three')
    plt.show()
    if save == True:
        print("save not implemented yet")